<a href="https://colab.research.google.com/github/silverstar0727/pragmatism-AI/blob/main/W%26B_keras_sweep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wandb -q

     |████████████████████████████████| 1.8MB 4.3MB/s 
     |████████████████████████████████| 102kB 13.8MB/s 
     |████████████████████████████████| 143kB 50.3MB/s 
     |████████████████████████████████| 143kB 50.1MB/s 
     |████████████████████████████████| 174kB 46.6MB/s 
     |████████████████████████████████| 71kB 9.4MB/s 
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.6 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.


In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.datasets import cifar10

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import wandb
from wandb.keras import WandbCallback

In [12]:
wandb.login()

True

In [43]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [44]:
class Model(tf.keras.Model):
    def __init__(self):
        super(Model, self).__init__()
        self.flatten = tf.keras.layers.Flatten(input_shape=(28, 28))
        self.linear1 = tf.keras.layers.Dense(128, activation='relu')
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.linear2 = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, x):
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.dropout(x)
        x = self.linear2(x)

        return x

In [45]:
def train():
    # Specify the hyperparameter to be tuned along with
    # an initial value
    config_defaults = {
        'batch_size': 8,
        'learning_rate': 0.01
    }

    # Initialize wandb with a sample project name
    wandb.init(config=config_defaults)

    # Specify the other hyperparameters to the configuration, if any
    wandb.config.epochs = 25

    # Prepare trainloader
    trainloader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    trainloader = trainloader.shuffle(1024).batch(wandb.config.batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    # prepare testloader 
    testloader = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    testloader = testloader.batch(wandb.config.batch_size).prefetch(tf.data.experimental.AUTOTUNE)

    # Iniialize model with hyperparameters
    keras.backend.clear_session()
    
    model = Model()

    # Compile the model
    opt = tf.keras.optimizers.Adam(learning_rate=wandb.config.learning_rate) # optimizer with different learning rate specified by config
    model.compile(opt, 'sparse_categorical_crossentropy', metrics=['acc'])
    
    # Train the model
    _ = model.fit(trainloader,
                  epochs=wandb.config.epochs, 
                  validation_data=testloader,
                  callbacks=[WandbCallback(save_model=False)]) # WandbCallback to automatically track metrics
                            
    # Evaluate    
    loss, accuracy = model.evaluate(testloader, callbacks=[WandbCallback(save_model=False)])
    print('Test Error Rate: ', round((1-accuracy)*100, 2))
    wandb.log({'Test Error Rate': round((1-accuracy)*100, 2)}) # wandb.log to track custom metrics

In [46]:
sweep_config = {
  'method': 'bayes', 
  'metric': {
      'name': 'val_loss',
      'goal': 'minimize'
  },
  'early_terminate':{
      'type': 'hyperband',
      'min_iter': 5
  },
  'parameters': {
      'batch_size': {
          'values': [8, 16, 32, 64, 128, 256]
      },
      'learning_rate':{
          'values': [0.01, 0.005, 0.001, 0.0005, 0.0001]
      }
  }
}

In [47]:
sweep_id = wandb.sweep(sweep_config, project="abc")

Create sweep with ID: qt9snm2s
Sweep URL: https://wandb.ai/jeongmin/abc/sweeps/qt9snm2s


In [48]:
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: jw6aqhx8 with config:
wandb: 	batch_size: 64
wandb: 	learning_rate: 0.0001


Epoch 1/25
938/938 [==============================] - 3s 3ms/step - loss: 0.8797 - acc: 0.7631 - val_loss: 0.3927 - val_acc: 0.9015
Epoch 2/25
938/938 [==============================] - 2s 2ms/step - loss: 0.3896 - acc: 0.8928 - val_loss: 0.2956 - val_acc: 0.9197
Epoch 3/25
938/938 [==============================] - 2s 2ms/step - loss: 0.3162 - acc: 0.9122 - val_loss: 0.2550 - val_acc: 0.9290
Epoch 4/25
938/938 [==============================] - 2s 2ms/step - loss: 0.2750 - acc: 0.9235 - val_loss: 0.2268 - val_acc: 0.9340
Epoch 5/25
938/938 [==============================] - 2s 2ms/step - loss: 0.2472 - acc: 0.9301 - val_loss: 0.2054 - val_acc: 0.9409
Epoch 6/25
938/938 [==============================] - 2s 3ms/step - loss: 0.2265 - acc: 0.9355 - val_loss: 0.1887 - val_acc: 0.9447
Epoch 7/25
938/938 [==============================] - 2s 2ms/step - loss: 0.2078 - acc: 0.9418 - val_loss: 0.1750 - val_acc: 0.9485
Epoch 8/25
938/938 [==============================] - 2s 2ms/step - loss: 0.

epoch,24
loss,0.08873
acc,0.97438
val_loss,0.08769
val_acc,0.9739
_runtime,75
_timestamp,1626693881
_step,25
best_val_loss,0.08769
best_epoch,24
Test Error Rate,2.61


epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
loss,█▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
acc,▁▅▆▆▇▇▇▇▇▇▇▇▇████████████
val_loss,█▆▅▄▄▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
val_acc,▁▃▄▄▅▅▆▆▆▆▇▇▇▇▇▇▇████████
_runtime,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▆▆▆▇▇▇▇███
_timestamp,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▆▆▆▇▇▇▇███
_step,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
Test Error Rate,▁


wandb: Agent Starting Run: 9g6xejgs with config:
wandb: 	batch_size: 128
wandb: 	learning_rate: 0.01


...

epoch,24
loss,0.02883
acc,0.99053
val_loss,0.06919
val_acc,0.9791
_runtime,31
_timestamp,1626693975
_step,25
best_val_loss,0.0658
best_epoch,17
Test Error Rate,2.09


epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
loss,█▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
acc,▁▅▆▆▇▇▇▇▇▇▇▇█████████████
val_loss,█▅▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▄▅▆▆▇▇▇▇▇▇██████████████
_runtime,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇▇███
_timestamp,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▆▆▆▆▇▇▇▇███
_step,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
Test Error Rate,▁


wandb: Agent Starting Run: j8q9apgo with config:
wandb: 	batch_size: 16
wandb: 	learning_rate: 0.001


Epoch 1/25
3750/3750 [==============================] - 8s 2ms/step - loss: 0.2561 - acc: 0.9262 - val_loss: 0.1293 - val_acc: 0.9604
Epoch 2/25
3750/3750 [==============================] - 8s 2ms/step - loss: 0.1258 - acc: 0.9619 - val_loss: 0.0945 - val_acc: 0.9715
Epoch 3/25
 615/3750 [===>..........................] - ETA: 6s - loss: 0.1016 - acc: 0.9695

wandb: Ctrl + C detected. Stopping sweep.
